<a href="https://colab.research.google.com/github/Hubajube/colab/blob/main/Utils_for_VQGan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section Links

*   [Mount GDrive](#gdrive)
*   [Copy all files to backup](#copystep)
*   [Copy selected files to backup](#copystep2)
*   [Video](#video)
*   [Parameters from Sheets](#sheetstest)
*   [Read stegano](#stegano)
*   [Count frames in video](#countframes)
*   [Slo-mo](#slomo)
*   [Extract frames](#videotoframes)
*   [Rename files](#rename)









In [ ]:
#@markdown #**Mount GDrive** 
#@markdown #<a name="gdrive"></a>
from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)

working_dir = '/content/gdrive/MyDrive/vqgan'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#@markdown #**Image libraries** 
#@markdown #<a name="imagelib"></a>

print("Installing libraries for handling metadata...")
!pip install stegano                                      &> /dev/null
!apt install exempi                                       &> /dev/null
!pip install python-xmp-toolkit                           &> /dev/null
!pip install imgtag                                       &> /dev/null
!pip install pillow==7.1.2                                &> /dev/null




In [1]:
#@markdown #**Make video**
#@markdown #<a name="video"></a>

from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)


runtype = "run from steps folder" #@param ["copy to video folder", "copy to video folder with numbers", "replace/add in video folder", "add to video folder", "run from video folder", "run from steps folder", "zoomed", "tests"]
startframe=19000#@param {type:"integer"} 
endframe=23070#@param {type:"integer"} 
fps=12#@param {type:"integer"} 
add_audio=True#@param {type:"boolean"} 

#copy files to video

#should delete files first!

from IPython.display import clear_output
from shutil import copyfile
import os

def copyfilewithnumber(source_file, target_file, number):
  #arial.ttf
  i=1
 
working_dir='/content/gdrive/MyDrive/vqgan'
if runtype=="run from steps folder":
  input_path = f'{working_dir}/steps'  
  output_path = f'{working_dir}/video'  
elif runtype=="zoomed":
  input_path = f'{working_dir}/zoomed'  
  output_path = f'{working_dir}/video'  
elif runtype=="tests":
  input_path = f'{working_dir}/tests'  
  output_path = f'{working_dir}/tests'  
else:
  input_path = f'{working_dir}/video'
  output_path = f'{working_dir}/video'

#if runtype in ["run from steps folder","run from video folder"]:
framecount = endframe - startframe + 1
#else:
#  framecount = 99999

video_name = f'{output_path}/video.mp4'
restart_frame=0
#fps=12

if runtype in ["copy to video folder","copy to video folder with numbers"]:
  !rm -r {input_path}
  !mkdir --parents {input_path}

if runtype in ["copy to video folder","replace/add in video folder"]:
  for f in range(startframe, endframe+1):
    #clear_output(wait=False)
    source_file = f'{working_dir}/steps/{f:05}.png'
    target_file = f'{working_dir}/video/{f:05}.png'
    print(source_file)
    copyfile(source_file, target_file)

if runtype == "copy to video folder with numbers":    
  for f in range(startframe, endframe+1):
    #clear_output(wait=False)
    source_file = f'{working_dir}/steps/{f:05}.png'
    target_file = f'{working_dir}/video/{f:05}.png'
    print(source_file)
    copyfilewithnumber(source_file, target_file, f)

if runtype == "add to video folder":
  for f in range(startframe, endframe+1):
    source_file = f'{working_dir}/steps/{f:05}.png'
    target_file = f'{working_dir}/video/{f:05}.png'
    if not(os.output_path.exists(target_file)):
      print(source_file)
      copyfile(source_file, target_file)

if runtype == "zoomed":
  image_name = f'{input_path}/zoomed_%05d.png'
else:
  image_name = f'{input_path}/%05d.png'

# import subprocess in case this cell is run without the above cells
import subprocess


frames = []



print(image_name)

cmd = [
    'ffmpeg',
    '-y',
    '-vcodec',
    'png',
    '-r',
    str(fps),
    '-start_number',
    str(startframe),
    '-i',
    image_name,
    '-frames:v',
    str(framecount),
    '-c:v',
    'libx264',
    '-vf',
    f'fps={fps}',
    '-pix_fmt',
    'yuv420p',
    '-crf',
    '17',
    '-preset',
    'veryslow',
    video_name
]

print(cmd)

process = subprocess.Popen(cmd, cwd=f'{input_path}/', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("The video is ready")

def frame_to_ss (frame):
  hours = int(frame/12/60/60)
  minutes = int(frame/12/60) - 60*hours
  seconds = int(frame/12) - 60*60*hours - 60*minutes
  milliseconds = round((frame - 60*60*12*hours - 60*12*minutes - 12*seconds) / 12 * 1000)

  ss = f'{hours:02d}:{minutes:02d}:{seconds:02d}.{milliseconds:03d}'
  print (ss)
  return ss


if add_audio:
  audio_name = f'{working_dir}/video/final.mp3'
  output_name = f'{output_path}/video_with_audio.mp4'

  if os.path.exists(output_name):
    print(f'Deleting {output_name}')
    os.remove(output_name)

  ss = frame_to_ss(startframe)

  cmd = [
        'ffmpeg',
        '-i',
        video_name,
        '-ss',
        ss,
        '-i',
        audio_name,
        '-map',
        '0:v:0',
        '-map',
        '1:a:0',
        '-c:v',
        'copy',
        '-c:a',
        'copy',
        '-shortest',
        output_name
    ]
  print(cmd)

  process = subprocess.Popen(cmd, cwd=f'{input_path}/', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  if process.returncode != 0:
      print(stderr)
      raise RuntimeError(stderr)
  else:
      print("The video is ready")

Mounted at /content/gdrive
/content/gdrive/MyDrive/vqgan/steps/%05d.png
['ffmpeg', '-y', '-vcodec', 'png', '-r', '12', '-start_number', '19000', '-i', '/content/gdrive/MyDrive/vqgan/steps/%05d.png', '-frames:v', '4071', '-c:v', 'libx264', '-vf', 'fps=12', '-pix_fmt', 'yuv420p', '-crf', '17', '-preset', 'veryslow', '/content/gdrive/MyDrive/vqgan/video/video.mp4']
The video is ready
Deleting /content/gdrive/MyDrive/vqgan/video/video_with_audio.mp4
00:26:23.333
['ffmpeg', '-i', '/content/gdrive/MyDrive/vqgan/video/video.mp4', '-ss', '00:26:23.333', '-i', '/content/gdrive/MyDrive/vqgan/video/final.mp3', '-map', '0:v:0', '-map', '1:a:0', '-c:v', 'copy', '-c:a', 'copy', '-shortest', '/content/gdrive/MyDrive/vqgan/video/video_with_audio.mp4']
The video is ready


In [ ]:
#@markdown #**Read stegano**
#@markdown #<a name="stegano"></a>

install_stegano=True#@param {type:"boolean"} 
filenumber=19534#@param {type:"integer"} 

if install_stegano:
  !pip install stegano                                      &> /dev/null
  !pip install pillow==7.1.2                                &> /dev/null


from stegano import lsb
import json


def get_stegano_data(filename):
  data = lsb.reveal(filename)
  j = json.loads(data)
  for t in j["title"].split("|"):
    if t.split(": ")[1].rstrip() != '0.0':
      print (t)
  print ()
  print (f'frame:{j["i"]}')
  print (f'model:{j["model"]}')
  print (f'seed:{j["seed"]}')


get_stegano_data(f"/content/gdrive/MyDrive/vqgan/steps/{filenumber}.png")



face: -5.0 
 anime: -5.0 
 flesh: -3.0 
 jungle planet: 4.234375 
 outer space: 2.53125 
 atmosphere: 3.8229166666666665 
 space ship: 0.17708333333333331 

frame:19534
model:S-FLCKR
seed:1


In [ ]:
#@markdown #**Count Frames in Video** 
#@markdown #<a name="countframes"></a>

video_folder = "/video/video.mp4" #@param ["/video.mp4", "/steps/video.mp4", "/video/video.mp4", "/video.mp4-slomo.mp4", "/steps/video.mp4-slomo.mp4", "/video/video.mp4-slomo.mp4"]
override_filename = "/tests/video.mp4" #@param {type:"string"}

if override_filename=="":
	video_filename=video_folder
else:
	video_filename=override_filename


# import the necessary packages
#from ..convenience import is_cv3
import cv2

def count_frames(path, override=False):
	# grab a pointer to the video file and initialize the total
	# number of frames read
	video = cv2.VideoCapture(path)
	total = 0
	# if the override flag is passed in, revert to the manual
	# method of counting frames
	if override:
		total = count_frames_manual(video)
	# otherwise, let's try the fast way first
	else:
		# lets try to determine the number of frames in a video
		# via video properties; this method can be very buggy
		# and might throw an error based on your OpenCV version
		# or may fail entirely based on your which video codecs
		# you have installed
		try:
			# check if we are using OpenCV 3
			if is_cv3():
				total = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
			# otherwise, we are using OpenCV 2.4
			else:
				total = int(video.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT))
		# uh-oh, we got an error -- revert to counting manually
		except:
			total = count_frames_manual(video)
	# release the video file pointer
	video.release()
	# return the total number of frames in the video
	return total

def count_frames_manual(video):
	# initialize the total number of frames read
	total = 0
	# loop over the frames of the video
	while True:
		# grab the current frame
		(grabbed, frame) = video.read()
	 
		# check to see if we have reached the end of the
		# video
		if not grabbed:
			break
		# increment the total number of frames read
		total += 1
	# return the total number of frames in the video file
	return total
f = count_frames(f'/content/gdrive/MyDrive/vqgan{video_filename}')
print(f)

In [ ]:
#@markdown #**Video to frames**
#@markdown #<a name="videotoframes"></a>

#ffmpeg -i file.mpg -r 1/1 $filename%03d.bmp

working_dir = '/content/gdrive/MyDrive/vqgan' 
video_name=f"{working_dir}/video/video.mp4-slomo.mp4"
input_path = f'{working_dir}/video'


cmd = [
    'ffmpeg',
    '-r',
    '1',
    '-i',
    video_name,
    "slomo1_%05d.png"
]

print(cmd)

process = subprocess.Popen(cmd, cwd=f'{input_path}/', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    print(stderr)
    raise RuntimeError(stderr)
else:
    print("The frames are ready")


In [ ]:
#@markdown #**Read from sheets**
#@markdown #<a name="sheetstest"></a>

from google.colab import drive
drive.mount('/content/gdrive')
#drive.mount("/content/gdrive", force_remount=True)

debugmode = False#@param {type:"boolean"} 

from google.colab import auth
auth.authenticate_user()


import gspread
from gspread import utils
#from gspread_formatting import *
from oauth2client.client import GoogleCredentials


def clean_worksheet(worksheet):
  framecell = worksheet.find('frame')
  framecellrow = framecell.row
  framecellcol = framecell.col
  ret = worksheet.get_all_values()

  #remove all the initial extra rows
  for row in range(1, framecellrow-1):
    ret.pop(0)
    #print(f'removing row: {ret.pop(0)}')

  #remove all the initial extra cols
  for rownum in range(0, len(ret)):
    for colnum in (0, framecellcol):
      ret[rownum].pop(0)
      #print(f'removing col: {ret[rownum].pop(0)}')

  #remove trailing rows
  rowcount = len(ret)
  for rownum in range(1,rowcount+1):
    if rownum>=rowcount:
      break
    if ret[rownum][0] == "":
      break

  for rowtodelete in range(rownum, rowcount):
    ret.pop(rownum)

  #remove trailing cols
  row = ret[1]
  colcount = len(row)
  for colnum in range(1,colcount+1):
    if colnum>=colcount:
      break
    if row[colnum] == "":
      break

  for coltodelete in range(colnum, colcount):
    for row in ret:
      row.pop(colnum)

  return ret

#def getweights(worksheet, rowstart, rowend, colnum):
#  weights = []
#  for rownum in range(rowstart, rowend+1):
#    weights.append(worksheet.cell(rownum, colnum).value)
#  return weights

def get_frames(allcells):
  ret = []
  for row in allcells[2:]:
    ret.append(row[0])
  return ret

def get_prompts(allcells, prompt_type):
  prompt_dict = {}
  colnum = 1
  for cell in allcells[1][1:]:
    if allcells[0][colnum] == prompt_type:
      frame_dict = {}
      for row in allcells[2:]:
        frame_dict[row[0]] = row[colnum]
      if prompt_type == 'img':
        prompt_name = f'/content/gdrive/MyDrive/vqgan/target/{allcells[1][colnum]}.png'
      else:
        prompt_name = allcells[1][colnum]
      prompt_dict[prompt_name] = frame_dict
    colnum = colnum + 1
  return prompt_dict


def fmt_parameter_weights(frames, parameters, parameter_name):
  fmt = ''
  for f,weights in parameters[parameter_name].items():
    if weights != '':
      fmt = fmt + f'{f}:({weights}),'
  return fmt[:-1]


def fmt_prompt_weights(frames, prompts):
  fmt = ''
  for f in frames:
    fmt = fmt + f'{f}:('
    lenprefix = len(f'{f}:(')
    hasweights = 0
    for p,weights in prompts.items():
      if weights[f] != '':
        fmt = fmt + f'{p}:{weights[f]}|'
        hasweights = 1
    if hasweights == 1:
      fmt = fmt[:-1] + '),'
    else:
      fmt = fmt[:-lenprefix]
  return fmt[:-1]

gc = gspread.authorize(GoogleCredentials.get_application_default())
spreadsheet = gc.open('VQGan')
allcells = clean_worksheet(spreadsheet.worksheet("VQGan"))



#get frames
frames = get_frames(allcells)


#get text and image prompts and other keyframed params
text_prompts = get_prompts(allcells, 'text')
image_prompts = get_prompts(allcells, 'img')
parameters = get_prompts(allcells, '')

if debugmode:
  print(f'allcells: {allcells}')
  print(f'frames: {frames}')
  print(f'text_prompts: {text_prompts}')
  print(f'image_prompts: {image_prompts}')
  print(f'parameters: {parameters}')

print(f'text_prompts: {fmt_prompt_weights(frames, text_prompts)}')
print(f'image_prompts: {fmt_prompt_weights(frames, image_prompts)}')
parameter_string = fmt_parameter_weights(frames, parameters, 'angle')
print(f'angle: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'zoom')
print(f'zoom: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'translate_x')
print(f'translate_x: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'translate_y')
print(f'translate_y: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'center_offset_x')
print(f'center_offset_x: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'center_offset_y')
print(f'center_offset_y: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'iterations')
print(f'iterations: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'vtilt')
print(f'vtilt: {parameter_string}')
parameter_string = fmt_parameter_weights(frames, parameters, 'htilt')
print(f'htilt: {parameter_string}')

from datetime import datetime

def writefmt(f, fmtstring):
    for item in fmtstring.split(','):
        f.write("%s\n" % item)
    f.write('\n')


def createlog():
    today = datetime.now().strftime("%Y%m%d%H%M%S")
    filename = f'test-{today}'

    #file1 = open(f"/content/gdrive/MyDrive/vqgan/logs/{filename}.log", "w") 
    #file1.write(f'text_prompts: {text_prompts}')
    with open(f"/content/gdrive/MyDrive/vqgan/logs/{filename}.log", "w") as f:
        #for item in fmt_prompt_weights(frames, text_prompts).split(','):
        #    f.write("%s\n" % item)
        f.write('text_prompts:\n')
        writefmt(f,fmt_prompt_weights(frames, text_prompts))
        f.write(f'{datetime.now()}\n')

createlog()


In [ ]:
#@markdown #**Slo-mo**
#@markdown #<a name="slomo"></a>


!git clone -q --depth 1 https://github.com/avinashpaliwal/Super-SloMo.git
from os.path import exists
def download_from_google_drive(file_id, file_name):
  # download a file from the Google Drive link
  !rm -f ./cookie
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id={file_id}" > /dev/null
  confirm_text = !awk '/download/ {print $NF}' ./cookie
  confirm_text = confirm_text[0]
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm={confirm_text}&id={file_id}" -o {file_name}
  
pretrained_model = 'SuperSloMo.ckpt'
if not exists(pretrained_model):
  download_from_google_drive('1IvobLDbRiBgZr3ryCRrWL8xDbMZ-KnpF', pretrained_model)


In [ ]:
#@markdown If the input video is X frames, then the output video is (X-1) * SLOW_MOTION_FACTOR frames. In other words, the final frame of the input will not be included in the output.

# import subprocess in case this cell is run without the above cells

import subprocess
import os

SLOW_MOTION_FACTOR = 8#@param {type:"number"}
TARGET_FPS = 12#@param {type:"number"}
extract_frames = True#@param {type:"boolean"}
filepath='/content/gdrive/MyDrive/vqgan/video/video.mp4'

if os.path.exists(f'{filepath}-slomo.mkv'):
  print(f'Deleting {filepath}-slomo.mkv')
  os.remove(f'{filepath}-slomo.mkv')

if os.path.exists(f'{filepath}-slomo.mp4'):
  print(f'Deleting {filepath}-slomo.mp4')
  os.remove(f'{filepath}-slomo.mp4')

cmd1 = [
    'python',
    'Super-SloMo/video_to_slomo.py',
    '--checkpoint',
    pretrained_model,
    '--video',
    filepath,
    '--sf',
    str(SLOW_MOTION_FACTOR),
    '--fps',
    str(TARGET_FPS),
    '--output',
    f'{filepath}-slomo.mkv',
]
process = subprocess.Popen(cmd1, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    raise RuntimeError(stderr)

cmd2 = [
    'ffmpeg',
    '-i',
    f'{filepath}-slomo.mkv',
    '-pix_fmt',
    'yuv420p',
    '-crf',
    '17',
    '-preset',
    'veryslow',
    f'{filepath}-slomo.mp4',
]

process = subprocess.Popen(cmd2, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()
if process.returncode != 0:
    raise RuntimeError(stderr)
    print(stderr)
    print(
        "You may be able to avoid this error by backing up the frames,"
        "restarting the notebook, and running only the video synthesis cells,"
        "or by decreasing the resolution of the image generation steps. "
        "If you restart the notebook, you will have to define the `filepath` manually"
        "by adding `filepath = 'PATH_TO_THE_VIDEO'` to the beginning of this cell. "
        "If these steps do not work, please post the traceback in the github."
    )

if extract_frames:
  cmd3 = [
      'ffmpeg',
      '-i',
      f'{filepath}-slomo.mkv',
      f'{filepath}\slomo\%05d.png',
  ]  
  process = subprocess.Popen(cmd2, cwd=f'/content', stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
  if process.returncode != 0:
      raise RuntimeError(stderr)
      print(stderr)
      print(
          "You may be able to avoid this error by backing up the frames,"
          "restarting the notebook, and running only the video synthesis cells,"
          "or by decreasing the resolution of the image generation steps. "
          "If you restart the notebook, you will have to define the `filepath` manually"
          "by adding `filepath = 'PATH_TO_THE_VIDEO'` to the beginning of this cell. "
          "If these steps do not work, please post the traceback in the github."
      )


In [ ]:
#@markdown #**Copy all files to backup**
# copy all steps to stepsbak
#@markdown #<a name="copystep"></a>
from shutil import copyfile
import os
 
os.chdir('/content/gdrive/MyDrive/vqgan/steps')
 
for k, f in enumerate(os.listdir()):
  print('/content/gdrive/MyDrive/vqgan/steps/'+f)
  copyfile('/content/gdrive/MyDrive/vqgan/steps/'+f, '/content/gdrive/MyDrive/vqgan/stepsbak/'+f)


In [ ]:
#@markdown #**Delete Files**
#@markdown #<a name="delfiles"></a>
import os
import fnmatch
 
os.chdir('/content/gdrive/MyDrive/vqgan/steps')

for k,f in enumerate(os.listdir()):
  if f[-3:]=='png' :
    if f[0]!='z':
      if f[0:3] > '7080':
        print(f)
  #copyfile('/content/gdrive/MyDrive/vqgan/steps/'+f, '/content/gdrive/MyDrive/vqgan/stepsbak/'+f)
        os.remove('/content/gdrive/MyDrive/vqgan/steps/'+f)




In [ ]:
#@markdown #**Rename files**
#@markdown #<a name="rename"></a>
import os
 
def RenameFile(oldfilename, newfilename):
  print(f'{oldfilename} -> {newfilename}')
  os.rename(oldfilename, newfilename)


for f in range(1, 2323):
  r = 14538 + f
  RenameFile(f'/content/gdrive/MyDrive/vqgan/slomo1_{f:05}.png', f'/content/gdrive/MyDrive/vqgan/{r:05}.png')


In [ ]:
# copy selected steps to stepsbak
#@markdown #**Copy selected files**
#@markdown #<a name="copystep2"></a>
from shutil import copyfile
import os
 
def CopyFile(filename, subfolder):
  sourcefolder = f'/content/gdrive/MyDrive/vqgan/steps'
  targetfolder = f'/content/gdrive/MyDrive/vqgan/{subfolder}'
  print(f'{sourcefolder}/{filename} -> {targetfolder}/{filename}')
  copyfile(f'{sourcefolder}/{filename}', f'{targetfolder}/{filename}')


#for f in range(12801, 14601):
for f in range(12803, 14600):
  CopyFile(f'{f:05}.png', 'video')
  #print(f'/content/gdrive/MyDrive/vqgan/steps/{f:05}.png')
  #copyfile(f'/content/gdrive/MyDrive/vqgan/steps/{f:05}.png', f'/content/gdrive/MyDrive/vqgan/stepsbak/{f:05}.png')
  #copyfile(f'/content/gdrive/MyDrive/vqgan/steps/{f:05}.png', f'/content/gdrive/MyDrive/vqgan/video/{f:05}.png')

